In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
log_dir = '../tgn/archive/day/log_hour'
log_path = log_dir + '/neg_sample/{}_{}_{}_self_sup.log'
method_list = ['JODIE', 'DyRep', 'TGN']
sample_list = ['rnd', 'hist_nre', 'induc_nre']
data_list = ['wikipedia', 'reddit', 'mooc', 'lastfm', 'enron', 'SocialEvo', 'uci']

In [3]:
def load_log(log_file):
    with open(log_file, 'r') as f_log:
        log_list = f_log.readlines()
    log_list = [log.strip() for log in log_list]
    return log_list

In [4]:
def parse_result(log_list):
    auc_list = []
    ap_list = []

    for log in log_list:
        if 'Old nodes -- auc_inherent' in log:
            auc_list.append(eval(log.split(':')[-1]))
        if 'Old nodes -- ap_inherent' in log:
            ap_list.append(eval(log.split(':')[-1]))   

    return auc_list, ap_list

In [5]:
for sample in sample_list:
    auc_result_list = []
    auc_std_result_list = []
    ap_result_list = []
    ap_std_result_list = []
    
    for method in method_list:
        auc_result_list.append([])
        auc_std_result_list.append([])
        ap_result_list.append([])
        ap_std_result_list.append([])
        valid_data_list = []
        
        for data in data_list:
            log_file = log_path.format(method, sample, data)
            if not os.path.exists(log_file):
                continue
            
            log_list = load_log(log_file)
            if len(log_list) <= 7:
                continue
            
            valid_data_list.append(data)
            auc_list, ap_list = parse_result(log_list)
            auc_mean, auc_std = np.mean(auc_list), np.std(auc_list)
            ap_mean, ap_std = np.mean(ap_list), np.std(ap_list)
            auc_result_list[-1].append(auc_mean)
            auc_std_result_list[-1].append(auc_std)
            ap_result_list[-1].append(ap_mean)
            ap_std_result_list[-1].append(ap_std)

    auc_df = pd.DataFrame(np.asarray(auc_result_list).T)
    auc_df.columns = method_list
    auc_df.index = valid_data_list
    auc_df.to_excel('{}/auc_{}.xlsx'.format(log_dir, sample))

    auc_std_df = pd.DataFrame(np.asarray(auc_std_result_list).T)
    auc_std_df.columns = method_list
    auc_std_df.index = valid_data_list
    auc_std_df.to_excel('{}/auc_std_{}.xlsx'.format(log_dir, sample))

    ap_df = pd.DataFrame(np.asarray(ap_result_list).T)
    ap_df.columns = method_list
    ap_df.index = valid_data_list
    ap_df.to_excel('{}/ap_{}.xlsx'.format(log_dir, sample))

    ap_std_df = pd.DataFrame(np.asarray(ap_std_result_list).T)
    ap_std_df.columns = method_list
    ap_std_df.index = valid_data_list
    ap_std_df.to_excel('{}/ap_std_{}.xlsx'.format(log_dir, sample))
    

In [6]:
auc_df.to_excel('{}/auc_{}.xlsx'.format(log_dir, sample))
auc_df

,JODIE,DyRep,TGN
wikipedia,0.437579,0.569212,0.700224
reddit,0.546280,0.469276,0.730804
mooc,0.263663,0.550611,0.454497
lastfm,0.333093,0.506959,0.512676
enron,0.494909,0.594608,0.670780
uci,0.451460,0.494683,0.641987
